In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/disaster-dataset/train.csv')


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords as nltk_stopwords

vectorizer = CountVectorizer()
stops = nltk_stopwords.words('english')

def process_texts(texts):
    texts = texts.lower()
    for x in [",", ":", "!", "?", ";", "[", "]", "(", ")", "\"", "\'",".", ",", "\"", "#", "@", "&"]:
        texts = texts.replace("x", "")
    texts = texts.split()
    cleaned_text = []
    for word in texts:
        if not (word in stops):
            cleaned_text.append(word)
    texts=""
    for word in cleaned_text:
        texts+=word+" "
    texts=texts.strip()
    return texts

bow = vectorizer.fit_transform(df["text"].apply(lambda x: process_texts(x)))
bow

In [ ]:
bow = bow.todense()
bow

In [ ]:
X = pd.DataFrame(bow)
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y, yt = train_test_split(X, df['target'], test_size=0.5, random_state=0)
X_train.head()

In [ ]:
X_test.head()

In [ ]:
y_train = np.array(y)
y_test = np.array(yt)

print(y_train, y_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gaus = GaussianNB()
gaus.fit(X_train, y_train)


In [ ]:
y_pred = gaus.predict(X_test)
print(y_pred)


In [ ]:
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_pred)

print('Average precision score: {0:0.2f}'.format(
      average_precision))

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt

disp = plot_precision_recall_curve(gaus, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

svc = LinearSVC(random_state=0, tol=1e-5)
svc.fit(X_train, y_train)


In [ ]:
y_pred_svc = svc.predict(X_test)
print(y_pred_svc)

In [ ]:
average_precision_svc = average_precision_score(y_test, y_pred_svc)

print('Average precision score: {0:0.2f}'.format(
      average_precision_svc))

In [ ]:
from sklearn.linear_model import LogisticRegression

log = LogisticRegression(random_state=0)
log.fit(X_train, y_train)

In [ ]:
y_pred_log = log.predict(X_test)
print(y_pred_log)

In [ ]:
average_precision_log = average_precision_score(y_test, y_pred_log)

print('Average precision score: {0:0.2f}'.format(
      average_precision_log))

In [ ]:
print('GaussianNB', average_precision)
print('LinearSVC', average_precision_svc)
print('LogisticRegression', average_precision_log)
